In [1]:
import numpy as np
import pandas as pd
import xgboost
from xgboost import XGBClassifier
from sklearn.cross_validation import train_test_split
from sklearn.metrics import f1_score
from sklearn import preprocessing

/usr/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
import sys
sys.path.insert(0, 'basic_codes')

import class_magnesium_not_drop_na #_transform_pairings
from class_magnesium_not_drop_na import *

In [3]:
fold = 'input_files/'
file_train = 'train.csv'
file_test = 'test.csv'

In [4]:
import importlib
importlib.reload(class_magnesium_not_drop_na)
from class_magnesium_not_drop_na import *

In [5]:
def load_data(filename, test=True):
    if test:
        data = pd.read_table(filename, sep=',').dropna()    
    else:
        data = pd.read_table(filename).dropna()
        
    data = data[~(data['chainlen']>1000)]

    if ('DSSR' in data.columns):
        data.drop('DSSR', axis=1, inplace=True)    
    
    features = list(deepcopy(data.columns))
    [features.remove(column) for column in ['Id','index', 'mg', 'pdb_chain'] if column in data.columns];
    x_test = np.array(data[features])
    
    try:
        y_test = np.array(data['mg'])
    except: 
        y_test = None
    change_output('Data loaded')
    return {'data':data, 'features':features, 'x':x_test, 'y':y_test}

In [6]:
test = load_data(fold+file_test, test=True)
test['data'].head()

Data loaded

,Id,index,pdb_chain,xray,resol,chainlen,protein,alpham2,betam2,gammam2,...,atomO2,atomO2p,atomO3p,atomO4,atomO4p,atomO5p,atomO6,atomOP1,atomOP2,atomOP3
0,0,28,1feu.cif1_F,1,2.3,21,1,-73.3,174.9,53.7,...,0,0,0,0,0,0,0,1,0,0
1,1,29,1feu.cif1_F,1,2.3,21,1,-73.3,174.9,53.7,...,0,0,0,0,0,0,0,0,1,0
2,2,30,1feu.cif1_F,1,2.3,21,1,-73.3,174.9,53.7,...,0,0,0,0,0,1,0,0,0,0
3,3,31,1feu.cif1_F,1,2.3,21,1,-73.3,174.9,53.7,...,0,0,0,0,1,0,0,0,0,0
4,4,32,1feu.cif1_F,1,2.3,21,1,-73.3,174.9,53.7,...,0,0,1,0,0,0,0,0,0,0


In [7]:
train = load_data(fold+file_train, test=False)
train['data'].head()

Data loaded

,pdb_chain,xray,resol,chainlen,protein,alpham2,betam2,gammam2,deltam2,epsilonm2,...,atomO2p,atomO3p,atomO4,atomO4p,atomO5p,atomO6,atomOP1,atomOP2,atomOP3,mg
35,1b23.cif1_R,1,2.6,74,1,-60.0,177.7,50.4,84.8,-150.6,...,0,0,0,0,0,0,1,0,0,0
36,1b23.cif1_R,1,2.6,74,1,-60.0,177.7,50.4,84.8,-150.6,...,0,0,0,0,0,0,0,1,0,0
37,1b23.cif1_R,1,2.6,74,1,-60.0,177.7,50.4,84.8,-150.6,...,0,0,0,0,1,0,0,0,0,0
38,1b23.cif1_R,1,2.6,74,1,-60.0,177.7,50.4,84.8,-150.6,...,0,0,0,1,0,0,0,0,0,0
39,1b23.cif1_R,1,2.6,74,1,-60.0,177.7,50.4,84.8,-150.6,...,0,1,0,0,0,0,0,0,0,0


In [8]:
unique, counts = np.unique(train['y'], return_counts=True)
dict(zip(unique, counts))
print(counts[0]/counts[1])

6.369751454257007


In [9]:
# model = XGBClassifier(max_depth=5, min_child_weight=4, learning_rate=0.1, objective='binary:logistic', scale_pos_weight=5,
#                       n_estimators=100)
# 0.36

In [10]:
model = XGBClassifier(max_depth=5, min_child_weight=4, learning_rate=0.1, objective='binary:logistic', scale_pos_weight=5,
                      n_estimators=100)
bst = model.fit(train['x'], train['y'])
# 0.39

In [11]:
importance = bst.feature_importances_
ind = np.argsort(importance)
good_ind = np.concatenate((ind[-10:], ind[:2]))
important_features = np.array(train['features'])[good_ind]
print(important_features)

['thetap1' 'thetap' 'moietyB' 'thetapp' 'thetapm1' 'etapp1' 'splaym1'
 'wtlen' 'resol' 'chainlen' 'SIIm1' 'SII']


In [13]:
# importance = bst.feature_importances_
# good_ind = [i for i, x in enumerate(importance) if x >= 0.01]
# important_features = np.array(train['features'])[good_ind]
# print(important_features)
# 0.36

In [12]:
for k in range(3):
    X_train, X_test, y_train, y_test = train_test_split(train['data'][important_features], train['y'], test_size=0.2)
    X_train = np.array(X_train)
    X_test = np.array(X_test)
    y_train = np.array(y_train)
    y_test = np.array(y_test)
    
    test_model = model.fit(X_train, y_train)
    y_pred = test_model.predict(X_test)
    print(f1_score(y_test, y_pred))

/usr/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


0.4066119691119692


/usr/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


0.4105301914580265
0.41783345615327927


/usr/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [14]:
import csv
def print_res(res_pred):
    with open("out.cvs", "w") as out_f:
        fieldnames = ['Id', 'mg']
        writer = csv.DictWriter(out_f, fieldnames=fieldnames)
        writer.writeheader()
        for i, ans in enumerate(res_pred):
            writer.writerow({'Id':i, 'mg':int(ans)})

In [15]:
def real_predict():
    bst = model.fit(train['data'][important_features], train['y'])
    preds = bst.predict(test['data'][important_features])
    return preds

In [16]:
preds = real_predict()

/usr/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [17]:
preds

array([0, 0, 0, ..., 0, 0, 0])

In [18]:
print_res(preds)

In [19]:
np.mean(preds)

0.12138442521631644

In [20]:
unique, counts = np.unique(preds, return_counts=True)
dict(zip(unique, counts))
counts[0]/counts[1]

7.238289205702648